In [1]:
import pandas as pd

# Load the datasets
brands_df = pd.read_json('d:/data/brands.json', lines=True)
receipts_df = pd.read_json('d:/data/receipts.json', lines=True)
users_df = pd.read_json('d:/data/users.json', lines=True)
print(brands_df) 
print(receipts_df)
print(users_df)

                                       _id       barcode            category  \
0     {'$oid': '601ac115be37ce2ead437551'}  511111019862              Baking   
1     {'$oid': '601c5460be37ce2ead43755f'}  511111519928           Beverages   
2     {'$oid': '601ac142be37ce2ead43755d'}  511111819905              Baking   
3     {'$oid': '601ac142be37ce2ead43755a'}  511111519874              Baking   
4     {'$oid': '601ac142be37ce2ead43755e'}  511111319917      Candy & Sweets   
...                                    ...           ...                 ...   
1162  {'$oid': '5f77274dbe37ce6b592e90c0'}  511111116752              Baking   
1163  {'$oid': '5dc1fca91dda2c0ad7da64ae'}  511111706328  Breakfast & Cereal   
1164  {'$oid': '5f494c6e04db711dd8fe87e7'}  511111416173      Candy & Sweets   
1165  {'$oid': '5a021611e4b00efe02b02a57'}  511111400608             Grocery   
1166  {'$oid': '6026d757be37ce6369301468'}  511111019930              Baking   

          categoryCode                 

In [5]:
import pandas as pd

# Load the datasets
brands_df = pd.read_json('d:/data/brands.json', lines=True)
receipts_df = pd.read_json('d:/data/receipts.json', lines=True)
users_df = pd.read_json('d:/data/users.json', lines=True)

# Function to convert unhashable columns to strings
def convert_unhashable_columns(df):
    for column in df.columns:
        # Check if the first element of the column is a list or dict, indicative of the entire column's type
        if df[column].apply(lambda x: isinstance(x, dict) or isinstance(x, list)).any():
            df[column] = df[column].apply(lambda x: str(x) if isinstance(x, dict) or isinstance(x, list) else x)
    return df


brands_df = convert_unhashable_columns(brands_df)
receipts_df = convert_unhashable_columns(receipts_df)
users_df = convert_unhashable_columns(users_df)
# Check for Duplicates
print("Brands duplicates:", brands_df.duplicated().sum())
print("Receipts duplicates:", receipts_df.duplicated().sum())
print("Users duplicates:", users_df.duplicated().sum())


Brands duplicates: 0
Receipts duplicates: 0
Users duplicates: 283


In [6]:
# Check for missing values

print("Brands missing values:\n", brands_df.isnull().sum(), "\n")
print("Receipts missing values:\n", receipts_df.isnull().sum(), "\n")
print("Users missing values:\n", users_df.isnull().sum(), "\n")

Brands missing values:
 _id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
dtype: int64 

Receipts missing values:
 _id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64 

Users missing values:
 _id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64 



In [13]:
print(users_df['createdDate'].head())
print(users_df['lastLogin'].head())

0    {'$date': 1609687444800}
1    {'$date': 1609687444800}
2    {'$date': 1609687444800}
3    {'$date': 1609687530554}
4    {'$date': 1609687444800}
Name: createdDate, dtype: object
0    {'$date': 1609687537858}
1    {'$date': 1609687537858}
2    {'$date': 1609687537858}
3    {'$date': 1609687530597}
4    {'$date': 1609687537858}
Name: lastLogin, dtype: object


In [19]:
def extract_date_ms(column):
    def extract(x):
        # Check if x is a dictionary, has '$date' key, and the value is not None
        if isinstance(x, dict) and '$date' in x and x['$date'] is not None:
            return x['$date']
        # Return None if x does not meet the criteria to avoid conversion errors
        return None
    return column.apply(extract)

# Apply the function to 'createdDate' and 'lastLogin', converting to datetime
users_df['createdDate'] = pd.to_datetime(extract_date_ms(users_df['createdDate']), unit='ms')
users_df['lastLogin'] = pd.to_datetime(extract_date_ms(users_df['lastLogin']), unit='ms')

# Find any rows where lastLogin is before createdDate
anomalies = users_df[users_df['lastLogin'] < users_df['createdDate']]
print("Anomalies where lastLogin is before createdDate:", len(anomalies))

# Displaying the first few rows for 'createdDate' and 'lastLogin' to verify
print(users_df['createdDate'].head())
print(users_df['lastLogin'].head())


Anomalies where lastLogin is before createdDate: 0
0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: createdDate, dtype: datetime64[ns]
0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: lastLogin, dtype: datetime64[ns]


In [20]:
# Detailed Data Quality Evaluation
valid_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
invalid_states_users = users_df[~users_df['state'].isin(valid_states)]
print(f"Users with invalid state codes: {len(invalid_states_users)}")


Users with invalid state codes: 56


In [22]:
# Numerical Data Outliers
receipts_df['totalSpent'].describe()


count     684.000000
mean       77.796857
std       347.110349
min         0.000000
25%         1.000000
50%        18.200000
75%        34.960000
max      4721.950000
Name: totalSpent, dtype: float64